This is my preprocessing pipeline to create unnested and seperated train dataframes which I can build models off. This notebook has nothing original it it. All the relevant code is taken from the starter notebook: https://www.kaggle.com/alokpattani/mlb-player-digital-engagement-data-exploration

The only reason I am releasing this notebook is that other people can run the PyTorch Baseline I published without problems. You can check it out here: https://www.kaggle.com/nicohrubec/mlb-pytorch-dnn-loop-feature-engineering/notebook

In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
train = pd.read_csv('../input/mlb-player-digital-engagement-forecasting/train.csv')

In [ ]:
def unnest_data(df, mode):
    # Get names of all "nested" data frames in daily training set
    daily_data_nested_df_names = df.drop('date', axis = 1).columns.values.tolist()

    for df_name in daily_data_nested_df_names:
        print("Processing ", df_name, " ...")
        date_nested_table = df[['date', df_name]]

        date_nested_table = (date_nested_table[
          ~pd.isna(date_nested_table[df_name])
          ].
          reset_index(drop = True)
          )

        daily_dfs_collection = []

        for date_index, date_row in date_nested_table.iterrows():
            daily_df = pd.read_json(date_row[df_name])

            daily_df['dailyDataDate'] = date_row['date']

            daily_dfs_collection = daily_dfs_collection + [daily_df]

        # Concatenate all daily dfs into single df for each row
        unnested_table = (pd.concat(daily_dfs_collection,
          ignore_index = True).
          # Set and reset index to move 'dailyDataDate' to front of df
          set_index('dailyDataDate').
          reset_index()
          )
        
        path = mode + '_' + df_name + '.csv'
        unnested_table.to_csv(path, index=False)

        # Clean up tables and collection of daily data frames for this df
        del(date_nested_table, daily_dfs_collection, unnested_table)

In [ ]:
unnest_data(train, 'train')